In [ ]:
# Install
!pip install datasets transformers huggingface_hub && apt-get install git-lfs

In [2]:
# Imports
from transformers import pipeline
import torch
torch.cuda.is_available()
from datasets import load_dataset
import numpy as np


In [3]:
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [4]:
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# For testing purposes, let's use a small subset
small_dataset = imdb["train"].shuffle(seed=120).select([i for i in list(range(5))])

In [6]:
small_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 5
})

In [7]:
small_dataset[-1]

{'text': "Shot into car from through the windscreen, someone is playing someone else their latest song, someone else didn't react, according to the voice-over. I just wonder how that came to be made. There were too many scenes in this movie that I wondered about how come a camera was there. If the scenes shot where the Warhols descended on a BJM post-party are true then that was inexcusable exploitation to the max, if not, then it was a total fabrication, either way it made me uncomfortable, if that was the purpose? All the way thru this movie I kept wondering how the footage came about. Taken at face value, a nice portrait of the (tortured) genius we all believe ourselves to be.",
 'label': 1}

In [8]:
len(small_dataset['text'])

5

In [9]:
classes = ['NEGATIVE', 'POSITIVE']
results = []
for review in small_dataset['text']:
  results.append(classes.index(sentiment_pipeline(review)[0]['label']))
results

[1, 0, 0, 1, 0]

In [10]:
y_true = np.array(small_dataset['label'])
x_true = np.array(small_dataset['text'])

In [11]:
# Compare to true labels
mismatches = np.where(results != y_true)[0]

# Return mismatched samples
X_mismatched = x_true[mismatches]
y_mismatched = y_true[mismatches]

print(f"There are {len(X_mismatched)} mismatches: {X_mismatched}")

There are 2 mismatches: ['"Meatball Machine" has got to be one of the most complex ridiculous, awful and over-exaggerated sci-fi horror films that I have ever came across. It is about good against evil and a coming-of-age tale, with the aim of to entertain with bloody, sleazy and humorous context. Because of that the violence isn\'t particularly gruesome and it doesn\'t make you squirm, but the gratuitous bloodletting and nudity does run freely. The performances by Issei Takahashi and Toru Tezuka is the worst i have seen, if that was not enough it is also directed by an unheard of director called Yudai Yamaguchi. This movie just have it all, it is bad to the bone!, A must see for every b-movie freak!!!... Simply: an enjoying and rare gem.'
 "Shot into car from through the windscreen, someone is playing someone else their latest song, someone else didn't react, according to the voice-over. I just wonder how that came to be made. There were too many scenes in this movie that I wondered a